In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM

In [0]:
CSV_PATH = "https://raw.githubusercontent.com/amingst/crypto-price-predictor/switch-to-colab-notebook/BTC-USD.csv"
FORECAST = 100
DROPOUT = 0.2
WINDOW_SIZE = FORECAST - 1
EPOCHS = 5
BATCH_SIZE = 64 

In [0]:
df = pd.read_csv(CSV_PATH, parse_dates=['Date'])
df = df.sort_values('Date')

In [0]:
def reshape(data_raw):
    min_max_scaler = MinMaxScaler()

    closing = data_raw.Close.values.reshape(-1, 1)
    closing_scaled = min_max_scaler.fit_transform(closing)
    closing_scaled = closing_scaled[~np.isnan(closing_scaled)]
    closing_scaled = closing_scaled.reshape(-1, 1)

    return closing_scaled

In [0]:
def to_time_series(data_raw, forecast_out):
  out = []
  
  for i in range(len(data_raw) - forecast_out):
    out.append(data_raw[i: i + forecast_out])
  
  return np.array(out)

In [0]:
def build_datasets(ts, forecast_out, split):
  train_size = int(split * ts.shape[0])

  x_train = ts[:train_size, :-1, :]
  y_train = ts[:train_size, -1, :]

  x_test = ts[:train_size, :-1, :]
  y_test = ts[:train_size, -1, :]

  return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = build_datasets(time_series, FORECAST, 0.95)

In [0]:
def build_model(w_size, dropout):
  model = Sequential()

  model.add(Bidirectional(LSTM(w_size, return_sequences=True), input_shape=(WINDOW_SIZE, x_train.shape[-1])))
  model.add(Dropout(rate=dropout))

  model.add(Bidirectional(LSTM((w_size * 2), return_sequences=True)))
  model.add(Dropout(rate=dropout))

  model.add(Bidirectional(LSTM(w_size, return_sequences=False)))
  model.add(Dense(units=1))
  model.add(Activation('linear'))

  return model

model = build_model(WINDOW_SIZE, DROPOUT)

In [0]:
model.compile(loss='mean_squared_error', optimizer='adam')

train_hist = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=False, validation_split=0.1)